# Option 1. Initialize without telemetry

* For first cluster

In [32]:
%%bash
terraform output -raw cluster1_init_remote
eval "$(terraform output -raw cluster1_init_remote)"

# Initialize Vault
export VAULT_ADDR=https://vault-eu-primary-europe-west1-ho5u.josemerchan-8d4c7e.gcp.sbx.hashicorpdemo.com:8200
export SKIP_TLS_VERIFY=true
vault status
curl -k $VAULT_ADDR

vault operator init -format=json > output.json
cat output.json | jq -r .root_token > root.token
export VAULT_TOKEN=$(cat root.token)
sleep 10

# Save info in GCP Secrets
gcloud secrets create root_token_europe-west1_ho5u --replication-policy="automatic" --project=hc-481920a3f7e54d39b33d0454ff9
echo -n $VAULT_TOKEN | gcloud secrets versions add root_token_europe-west1_ho5u --project=hc-481920a3f7e54d39b33d0454ff9 --data-file=-
gcloud secrets create vault_init_data_europe-west1_ho5u --replication-policy="automatic" --project=hc-481920a3f7e54d39b33d0454ff9
cat output.json | gcloud secrets versions add vault_init_data_europe-west1_ho5u --project=hc-481920a3f7e54d39b33d0454ff9 --data-file=-
rm output.json
rm root.token

# Enable Audit Logging
vault audit enable file file_path=/var/log/vault.log

# Enab

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    40  100    40    0     0    359      0 --:--:-- --:--:-- --:--:--   388


<a href="/ui/">Temporary Redirect</a>.



Created secret [root_token_europe-west1_ho5u].
Created version [1] of the secret [root_token_europe-west1_ho5u].
Created secret [vault_init_data_europe-west1_ho5u].
Created version [1] of the secret [vault_init_data_europe-west1_ho5u].


Success! Enabled the file audit device at: file/


created key [eddcc57f630a71504a08f84781734a5c30b2c679] of type [json] as [sa-keys__europe-west1_ho5u.json] for [iam-vault-ho5u@hc-481920a3f7e54d39b33d0454ff9.iam.gserviceaccount.com]


Success! Data written to: sys/storage/raft/snapshot-auto/config/hourly


* For second cluster

In [33]:
%%bash
terraform output -raw cluster2_init_remote
eval "$(terraform output -raw cluster2_init_remote)"

# Initialize Vault
export VAULT_ADDR=https://vault-eu-secondary-europe-west1-p3mi.josemerchan-8d4c7e.gcp.sbx.hashicorpdemo.com:8200
export SKIP_TLS_VERIFY=true
vault status
curl -k $VAULT_ADDR

vault operator init -format=json > output.json
cat output.json | jq -r .root_token > root.token
export VAULT_TOKEN=$(cat root.token)
sleep 10

# Save info in GCP Secrets
gcloud secrets create root_token_europe-west1_p3mi --replication-policy="automatic" --project=hc-481920a3f7e54d39b33d0454ff9
echo -n $VAULT_TOKEN | gcloud secrets versions add root_token_europe-west1_p3mi --project=hc-481920a3f7e54d39b33d0454ff9 --data-file=-
gcloud secrets create vault_init_data_europe-west1_p3mi --replication-policy="automatic" --project=hc-481920a3f7e54d39b33d0454ff9
cat output.json | gcloud secrets versions add vault_init_data_europe-west1_p3mi --project=hc-481920a3f7e54d39b33d0454ff9 --data-file=-
rm output.json
rm root.token

# Enable Audit Logging
vault audit enable file file_path=/var/log/vault.log

# En

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    40  100    40    0     0    377      0 --:--:-- --:--:-- --:--:--   388


<a href="/ui/">Temporary Redirect</a>.



Created secret [root_token_europe-west1_p3mi].
Created version [1] of the secret [root_token_europe-west1_p3mi].
Created secret [vault_init_data_europe-west1_p3mi].
Created version [1] of the secret [vault_init_data_europe-west1_p3mi].


Success! Enabled the file audit device at: file/


created key [5df85e3ed1485a49d8aaad5bb4c69efef298b72f] of type [json] as [sa-keys__europe-west1_p3mi.json] for [iam-vault-p3mi@hc-481920a3f7e54d39b33d0454ff9.iam.gserviceaccount.com]


Success! Data written to: sys/storage/raft/snapshot-auto/config/hourly


# Option 2. Initialize with telemetry

* Initialize the first cluster. Requires SSH access via console.
* Enable Audit logging
* Obtain a token and configure the agent to use it

In [ ]:
%%bash
terraform output -raw cluster1_init_script_node1

* Obtain token for Ops Agent
* Restart Ops Agent Service
* Repeat on nodes 2 to X

In [ ]:
%%bash
terraform output -raw cluster1_init_script_node2-X

* Enable Automatic Backups on local terminal

In [ ]:
%%bash
terraform output -raw cluster1_init_auto_snapshot

* Connect to cluster

In [ ]:
%%bash
terraform output -raw cluster1_fqdn_8200

* Repeat for other clusters

In [ ]:
%%bash
terraform output -raw cluster2_init_script_node1

In [34]:
%%bash
terraform output -raw cluster2_init_script_node2-X


# Obtain root token
export VAULT_TOKEN=$(gcloud secrets versions access latest --secret=root_token_europe-west1_p3mi)
vault status # wait for vault to show as unsealed

# Create Prometheus token and update Ops Agent Configuration with Token
vault token create -field=token -policy prometheus-metrics > prometheus.token
export PROMETHEUS_TOKEN=$(cat prometheus.token)
sudo sed -ibak "s/VAULT_TOKEN_PROMETHEUS/$PROMETHEUS_TOKEN/g" /etc/google-cloud-ops-agent/config.yaml
sudo systemctl restart google-cloud-ops-agent



In [ ]:
%%bash
terraform output -raw cluster2_init_auto_snapshot

* Connect to cluster

In [ ]:
%%bash
terraform output -raw cluster2_fqdn_8200